# Lab 7: Object Detection - Part 2
## NMS, Fine-tuning, and Evaluation

**Duration**: ~3 hours

### Learning Objectives
By the end of this lab, you will be able to:
1. Understand and implement Non-Maximum Suppression (NMS)
2. Prepare custom datasets for YOLO training
3. Fine-tune YOLO on your own data
4. Evaluate detection models with mAP

### Prerequisites
- Completed Lab 6 (Object Detection basics)

### GPU Recommendation
Fine-tuning YOLO benefits significantly from GPU. We recommend using Google Colab with GPU for Part 3.

In [ ]:
# Install required packages
# !pip install ultralytics supervision opencv-python matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import cv2
import os
import torch

from ultralytics import YOLO
import supervision as sv

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

---

# Part 1: Non-Maximum Suppression (NMS)

## The Problem: Duplicate Detections

Object detectors often output multiple overlapping boxes for the same object:

```
BEFORE NMS:                    AFTER NMS:

┌─────────────┐               ┌─────────────┐
│┌───────────┐│               │             │
││┌─────────┐││               │    🐕       │
│││  🐕     │││       →       │   (0.95)    │
│││ (0.95)  │││               │             │
││└─────────┘││               └─────────────┘
│└───────────┘│
└─────────────┘

Multiple boxes              One best box
(0.95, 0.90, 0.85)         (0.95)
```

**NMS keeps the best box and removes overlapping duplicates.**

In [ ]:
# SOLVED EXAMPLE: IoU Calculation (from Lab 6)

def calculate_iou(box1, box2):
    """Calculate IoU between two boxes in xyxy format."""
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2
    
    x1_i = max(x1_1, x1_2)
    y1_i = max(y1_1, y1_2)
    x2_i = min(x2_1, x2_2)
    y2_i = min(y2_1, y2_2)
    
    if x2_i < x1_i or y2_i < y1_i:
        intersection = 0
    else:
        intersection = (x2_i - x1_i) * (y2_i - y1_i)
    
    area1 = (x2_1 - x1_1) * (y2_1 - y1_1)
    area2 = (x2_2 - x1_2) * (y2_2 - y1_2)
    union = area1 + area2 - intersection
    
    return intersection / union if union > 0 else 0

In [ ]:
# SOLVED EXAMPLE: NMS Algorithm

def nms(boxes, scores, iou_threshold=0.5):
    """
    Non-Maximum Suppression.
    
    Args:
        boxes: List of boxes in xyxy format
        scores: List of confidence scores
        iou_threshold: Remove boxes with IoU > threshold
    
    Returns:
        List of indices to keep
    """
    if len(boxes) == 0:
        return []
    
    # Sort by confidence (descending)
    sorted_indices = np.argsort(scores)[::-1]
    
    keep = []
    
    while len(sorted_indices) > 0:
        # Pick the box with highest confidence
        best_idx = sorted_indices[0]
        keep.append(best_idx)
        
        # Compare with remaining boxes
        remaining_indices = []
        for idx in sorted_indices[1:]:
            iou = calculate_iou(boxes[best_idx], boxes[idx])
            
            # Keep if IoU is below threshold (not overlapping too much)
            if iou < iou_threshold:
                remaining_indices.append(idx)
        
        sorted_indices = remaining_indices
    
    return keep

# Test
boxes = [
    (100, 100, 200, 200),  # Box 0
    (110, 110, 210, 210),  # Box 1 (overlaps with 0)
    (105, 105, 205, 205),  # Box 2 (overlaps with 0 and 1)
    (300, 300, 400, 400),  # Box 3 (separate)
]
scores = [0.95, 0.90, 0.85, 0.80]

keep_indices = nms(boxes, scores, iou_threshold=0.5)
print(f"Original boxes: {len(boxes)}")
print(f"Kept indices: {keep_indices}")
print(f"Boxes after NMS: {len(keep_indices)}")

In [ ]:
# SOLVED EXAMPLE: Visualize NMS

def visualize_nms(boxes, scores, keep_indices):
    """Visualize before and after NMS."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    colors = plt.cm.tab10.colors
    
    # Before NMS
    for i, (box, score) in enumerate(zip(boxes, scores)):
        rect = patches.Rectangle(
            (box[0], box[1]), box[2]-box[0], box[3]-box[1],
            linewidth=2, edgecolor=colors[i], facecolor=colors[i], alpha=0.3,
            label=f'Box {i}: {score:.2f}'
        )
        axes[0].add_patch(rect)
    
    axes[0].set_xlim(0, 500)
    axes[0].set_ylim(500, 0)
    axes[0].set_title(f'Before NMS ({len(boxes)} boxes)')
    axes[0].legend(loc='upper right')
    axes[0].set_aspect('equal')
    
    # After NMS
    for i in keep_indices:
        box = boxes[i]
        score = scores[i]
        rect = patches.Rectangle(
            (box[0], box[1]), box[2]-box[0], box[3]-box[1],
            linewidth=3, edgecolor=colors[i], facecolor=colors[i], alpha=0.4,
            label=f'Box {i}: {score:.2f}'
        )
        axes[1].add_patch(rect)
    
    axes[1].set_xlim(0, 500)
    axes[1].set_ylim(500, 0)
    axes[1].set_title(f'After NMS ({len(keep_indices)} boxes)')
    axes[1].legend(loc='upper right')
    axes[1].set_aspect('equal')
    
    plt.tight_layout()
    plt.show()

visualize_nms(boxes, scores, keep_indices)

In [ ]:
# SOLVED EXAMPLE: NMS Threshold Effect

# More overlapping boxes for demonstration
boxes = [
    (100, 100, 200, 200),
    (110, 110, 210, 210),
    (120, 120, 220, 220),
    (130, 130, 230, 230),
    (300, 100, 400, 200),
    (305, 105, 405, 205),
]
scores = [0.95, 0.90, 0.85, 0.80, 0.75, 0.70]

thresholds = [0.3, 0.5, 0.7, 0.9]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for ax, thresh in zip(axes, thresholds):
    keep = nms(boxes, scores, iou_threshold=thresh)
    
    colors = plt.cm.tab10.colors
    for i in keep:
        box = boxes[i]
        rect = patches.Rectangle(
            (box[0], box[1]), box[2]-box[0], box[3]-box[1],
            linewidth=2, edgecolor=colors[i], facecolor=colors[i], alpha=0.4
        )
        ax.add_patch(rect)
    
    ax.set_xlim(50, 450)
    ax.set_ylim(250, 50)
    ax.set_title(f'IoU thresh = {thresh}\n({len(keep)} boxes kept)')
    ax.set_aspect('equal')

plt.suptitle('Effect of NMS IoU Threshold', fontsize=14)
plt.tight_layout()
plt.show()

print("Lower threshold = More aggressive removal")
print("Higher threshold = Keep more overlapping boxes")

## Question 1.1

Implement Class-Aware NMS: Only suppress boxes if they have the same class.

Hint: Group boxes by class first, then apply NMS to each group.

In [ ]:
# YOUR CODE HERE

def class_aware_nms(boxes, scores, classes, iou_threshold=0.5):
    """
    NMS that only suppresses boxes of the same class.
    
    Args:
        boxes: List of boxes
        scores: List of confidence scores
        classes: List of class IDs
        iou_threshold: IoU threshold
    
    Returns:
        List of indices to keep
    """
    pass

# Test
# boxes = [(100, 100, 200, 200), (110, 110, 210, 210), (100, 100, 200, 200)]
# scores = [0.9, 0.8, 0.85]
# classes = [0, 0, 1]  # First two are same class, third is different
# Expected: Keep 0 and 2 (or 1 and 2)

---

# Part 2: Custom Dataset Preparation

## YOLO Dataset Format

YOLO expects data in a specific format:

```
dataset/
├── images/
│   ├── train/
│   │   ├── img001.jpg
│   │   └── img002.jpg
│   └── val/
│       ├── img003.jpg
│       └── img004.jpg
├── labels/
│   ├── train/
│   │   ├── img001.txt
│   │   └── img002.txt
│   └── val/
│       ├── img003.txt
│       └── img004.txt
└── data.yaml
```

**Label format (one line per object):**
```
class_id x_center y_center width height
```

All values are normalized (0-1)!

In [ ]:
# SOLVED EXAMPLE: Convert Coordinates to YOLO Format

def xyxy_to_yolo(box, img_width, img_height):
    """
    Convert xyxy box to YOLO format (normalized cxcywh).
    
    Args:
        box: (x1, y1, x2, y2) in pixels
        img_width, img_height: Image dimensions
    
    Returns:
        (x_center, y_center, width, height) normalized 0-1
    """
    x1, y1, x2, y2 = box
    
    # Calculate center and size
    x_center = (x1 + x2) / 2 / img_width
    y_center = (y1 + y2) / 2 / img_height
    width = (x2 - x1) / img_width
    height = (y2 - y1) / img_height
    
    return (x_center, y_center, width, height)

def yolo_to_xyxy(yolo_box, img_width, img_height):
    """
    Convert YOLO format back to xyxy (pixels).
    """
    x_center, y_center, width, height = yolo_box
    
    x1 = (x_center - width/2) * img_width
    y1 = (y_center - height/2) * img_height
    x2 = (x_center + width/2) * img_width
    y2 = (y_center + height/2) * img_height
    
    return (x1, y1, x2, y2)

# Example
img_w, img_h = 640, 480
box_xyxy = (100, 50, 300, 200)

yolo_format = xyxy_to_yolo(box_xyxy, img_w, img_h)
print(f"Original (xyxy): {box_xyxy}")
print(f"YOLO format: ({yolo_format[0]:.4f}, {yolo_format[1]:.4f}, {yolo_format[2]:.4f}, {yolo_format[3]:.4f})")

# Convert back
back_to_xyxy = yolo_to_xyxy(yolo_format, img_w, img_h)
print(f"Back to xyxy: ({back_to_xyxy[0]:.0f}, {back_to_xyxy[1]:.0f}, {back_to_xyxy[2]:.0f}, {back_to_xyxy[3]:.0f})")

In [ ]:
# SOLVED EXAMPLE: Create a YOLO Label File

def create_yolo_label(annotations, img_width, img_height):
    """
    Create YOLO label file content.
    
    Args:
        annotations: List of (class_id, x1, y1, x2, y2)
        img_width, img_height: Image dimensions
    
    Returns:
        String content for .txt label file
    """
    lines = []
    for ann in annotations:
        class_id = ann[0]
        box = ann[1:5]
        yolo_box = xyxy_to_yolo(box, img_width, img_height)
        
        line = f"{class_id} {yolo_box[0]:.6f} {yolo_box[1]:.6f} {yolo_box[2]:.6f} {yolo_box[3]:.6f}"
        lines.append(line)
    
    return "\n".join(lines)

# Example annotations: [(class_id, x1, y1, x2, y2), ...]
annotations = [
    (0, 100, 50, 300, 200),   # Class 0: dog
    (1, 350, 100, 500, 300),  # Class 1: cat
    (0, 50, 250, 150, 400),   # Class 0: another dog
]

label_content = create_yolo_label(annotations, 640, 480)
print("YOLO label file content:")
print(label_content)

In [ ]:
# SOLVED EXAMPLE: Create data.yaml Configuration

def create_data_yaml(train_path, val_path, class_names):
    """
    Create YOLO data.yaml configuration.
    """
    yaml_content = f"""# YOLO Dataset Configuration
path: .  # Dataset root directory
train: {train_path}  # Training images
val: {val_path}  # Validation images

# Classes
names:
"""
    for i, name in enumerate(class_names):
        yaml_content += f"  {i}: {name}\n"
    
    return yaml_content

# Example
class_names = ['dog', 'cat', 'bird']
yaml_content = create_data_yaml('images/train', 'images/val', class_names)

print("data.yaml content:")
print(yaml_content)

## Using Roboflow for Easy Labeling

[Roboflow](https://roboflow.com/) is a free tool for:
- Labeling images with bounding boxes
- Augmenting your dataset
- Exporting in YOLO format

For this lab, we'll use a pre-made dataset from Roboflow Universe.

In [ ]:
# SOLVED EXAMPLE: Download Sample Dataset from Roboflow

# We'll use a small playing cards dataset
# You can browse more at: https://universe.roboflow.com/

from roboflow import Roboflow

# Note: For the lab, we'll create a minimal synthetic dataset
# In practice, you'd download from Roboflow like this:
# rf = Roboflow(api_key="YOUR_API_KEY")
# project = rf.workspace("workspace").project("project-name")
# dataset = project.version(1).download("yolov8")

print("For a real project, sign up at roboflow.com and download a dataset!")
print("\nWe'll create a synthetic dataset for demonstration.")

In [ ]:
# SOLVED EXAMPLE: Create Synthetic Dataset for Demo

import shutil

# Create directory structure
dataset_dir = "demo_dataset"
os.makedirs(f"{dataset_dir}/images/train", exist_ok=True)
os.makedirs(f"{dataset_dir}/images/val", exist_ok=True)
os.makedirs(f"{dataset_dir}/labels/train", exist_ok=True)
os.makedirs(f"{dataset_dir}/labels/val", exist_ok=True)

# Create synthetic images with colored rectangles (simulating objects)
np.random.seed(42)

class_colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]  # RGB
class_names = ['red_square', 'green_square', 'blue_square']

def create_synthetic_image(img_size=(416, 416), num_objects=3):
    """Create a synthetic image with colored squares."""
    img = np.ones((img_size[1], img_size[0], 3), dtype=np.uint8) * 200  # Gray background
    annotations = []
    
    for _ in range(num_objects):
        # Random class
        class_id = np.random.randint(0, 3)
        color = class_colors[class_id]
        
        # Random position and size
        size = np.random.randint(30, 80)
        x1 = np.random.randint(0, img_size[0] - size)
        y1 = np.random.randint(0, img_size[1] - size)
        x2 = x1 + size
        y2 = y1 + size
        
        # Draw rectangle
        cv2.rectangle(img, (x1, y1), (x2, y2), color, -1)
        
        annotations.append((class_id, x1, y1, x2, y2))
    
    return img, annotations

# Create training images
for i in range(20):
    img, annotations = create_synthetic_image(num_objects=np.random.randint(1, 5))
    
    # Save image
    img_path = f"{dataset_dir}/images/train/img_{i:03d}.jpg"
    cv2.imwrite(img_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    
    # Save label
    label_content = create_yolo_label(annotations, 416, 416)
    label_path = f"{dataset_dir}/labels/train/img_{i:03d}.txt"
    with open(label_path, 'w') as f:
        f.write(label_content)

# Create validation images
for i in range(5):
    img, annotations = create_synthetic_image(num_objects=np.random.randint(1, 5))
    
    img_path = f"{dataset_dir}/images/val/img_{i:03d}.jpg"
    cv2.imwrite(img_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    
    label_content = create_yolo_label(annotations, 416, 416)
    label_path = f"{dataset_dir}/labels/val/img_{i:03d}.txt"
    with open(label_path, 'w') as f:
        f.write(label_content)

# Create data.yaml
yaml_content = f"""path: {os.path.abspath(dataset_dir)}
train: images/train
val: images/val

names:
  0: red_square
  1: green_square
  2: blue_square
"""

with open(f"{dataset_dir}/data.yaml", 'w') as f:
    f.write(yaml_content)

print("Created synthetic dataset:")
print(f"  Training images: 20")
print(f"  Validation images: 5")
print(f"  Classes: {class_names}")

In [ ]:
# SOLVED EXAMPLE: Visualize Dataset

# Load a sample image and its labels
sample_img = cv2.imread(f"{dataset_dir}/images/train/img_000.jpg")
sample_img = cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB)

with open(f"{dataset_dir}/labels/train/img_000.txt", 'r') as f:
    labels = f.read().strip().split('\n')

# Draw boxes
h, w = sample_img.shape[:2]
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]

for label in labels:
    parts = label.split()
    class_id = int(parts[0])
    yolo_box = [float(x) for x in parts[1:]]
    
    # Convert to xyxy
    x1, y1, x2, y2 = yolo_to_xyxy(yolo_box, w, h)
    
    cv2.rectangle(sample_img, (int(x1), int(y1)), (int(x2), int(y2)), colors[class_id], 2)
    cv2.putText(sample_img, class_names[class_id], (int(x1), int(y1)-5), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[class_id], 2)

plt.figure(figsize=(8, 8))
plt.imshow(sample_img)
plt.title('Sample Training Image with Annotations')
plt.axis('off')
plt.show()

## Question 2.1

Write a function that validates a YOLO label file:
1. Check that all coordinates are between 0 and 1
2. Check that class IDs are valid
3. Check that boxes have positive width and height

In [ ]:
# YOUR CODE HERE

def validate_yolo_label(label_path, num_classes):
    """
    Validate a YOLO label file.
    
    Returns:
        (is_valid, errors): Tuple of boolean and list of error messages
    """
    pass

---

# Part 3: Fine-tuning YOLO

## Why Fine-tune?

Pre-trained YOLO knows 80 common objects (COCO dataset).

But what if you want to detect:
- Specific product types
- Custom machinery parts
- Rare objects not in COCO

**Fine-tuning = Start from pre-trained weights, train on YOUR data**

In [ ]:
# SOLVED EXAMPLE: Fine-tune YOLO on Custom Data

# Load pre-trained model
model = YOLO('yolov8n.pt')

# Fine-tune on our dataset
# Note: This will take a few minutes even on GPU
results = model.train(
    data=f"{dataset_dir}/data.yaml",
    epochs=10,        # More epochs for better results
    imgsz=416,        # Image size
    batch=8,          # Batch size (reduce if OOM)
    patience=5,       # Early stopping
    verbose=True,
    plots=True,       # Generate training plots
)

In [ ]:
# SOLVED EXAMPLE: Evaluate Fine-tuned Model

# Load the best model
best_model = YOLO('runs/detect/train/weights/best.pt')

# Validate on validation set
metrics = best_model.val()

print("\nValidation Metrics:")
print(f"  mAP50: {metrics.box.map50:.4f}")
print(f"  mAP50-95: {metrics.box.map:.4f}")

In [ ]:
# SOLVED EXAMPLE: Test on New Image

# Create a test image
test_img, test_annotations = create_synthetic_image(num_objects=4)

# Run detection
results = best_model(test_img)
result = results[0]

# Visualize
annotated = result.plot()
annotated = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10, 10))
plt.imshow(annotated)
plt.title('Detection with Fine-tuned Model')
plt.axis('off')
plt.show()

---

# Part 4: Evaluation Metrics

## Precision and Recall

```
PRECISION: Of all detections, how many are correct?
           Precision = TP / (TP + FP)

RECALL: Of all ground truth objects, how many did we find?
        Recall = TP / (TP + FN)

Where:
  TP (True Positive)  = Correct detection (IoU >= 0.5)
  FP (False Positive) = Wrong detection (no matching GT)
  FN (False Negative) = Missed object (GT with no detection)
```

In [ ]:
# SOLVED EXAMPLE: Calculate Precision and Recall

def calculate_metrics(pred_boxes, gt_boxes, iou_threshold=0.5):
    """
    Calculate precision and recall.
    
    Args:
        pred_boxes: List of predicted boxes
        gt_boxes: List of ground truth boxes
        iou_threshold: IoU threshold for correct detection
    
    Returns:
        (precision, recall, tp, fp, fn)
    """
    tp = 0
    fp = 0
    matched_gt = set()
    
    for pred in pred_boxes:
        best_iou = 0
        best_gt_idx = -1
        
        for gt_idx, gt in enumerate(gt_boxes):
            if gt_idx in matched_gt:
                continue
            iou = calculate_iou(pred, gt)
            if iou > best_iou:
                best_iou = iou
                best_gt_idx = gt_idx
        
        if best_iou >= iou_threshold:
            tp += 1
            matched_gt.add(best_gt_idx)
        else:
            fp += 1
    
    fn = len(gt_boxes) - len(matched_gt)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    return precision, recall, tp, fp, fn

# Example
predictions = [(100, 100, 200, 200), (300, 300, 400, 400), (50, 50, 100, 100)]
ground_truths = [(105, 105, 205, 205), (310, 310, 410, 410)]  # Only 2 objects

prec, rec, tp, fp, fn = calculate_metrics(predictions, ground_truths)

print("Metrics:")
print(f"  True Positives: {tp}")
print(f"  False Positives: {fp}")
print(f"  False Negatives: {fn}")
print(f"  Precision: {prec:.2%}")
print(f"  Recall: {rec:.2%}")

## Mean Average Precision (mAP)

mAP is the standard metric for object detection.

```
mAP CALCULATION:

1. For each class:
   - Sort detections by confidence
   - Calculate precision/recall at each threshold
   - Compute Average Precision (area under P-R curve)

2. Average across all classes = mAP

mAP@50: IoU threshold = 0.5
mAP@50:95: Average mAP at IoU from 0.5 to 0.95 (step 0.05)
```

In [ ]:
# SOLVED EXAMPLE: Precision-Recall Curve

def precision_recall_curve(predictions, ground_truths, iou_threshold=0.5):
    """
    Calculate precision-recall curve.
    
    Args:
        predictions: List of (box, confidence)
        ground_truths: List of boxes
    
    Returns:
        precisions, recalls
    """
    # Sort by confidence
    sorted_preds = sorted(predictions, key=lambda x: x[1], reverse=True)
    
    precisions = []
    recalls = []
    
    for i in range(1, len(sorted_preds) + 1):
        boxes = [p[0] for p in sorted_preds[:i]]
        prec, rec, _, _, _ = calculate_metrics(boxes, ground_truths, iou_threshold)
        precisions.append(prec)
        recalls.append(rec)
    
    return precisions, recalls

# Example with varying confidence
predictions_with_conf = [
    ((100, 100, 200, 200), 0.95),  # Good detection
    ((105, 105, 205, 205), 0.85),  # Duplicate of first
    ((300, 300, 400, 400), 0.75),  # Good detection
    ((500, 500, 600, 600), 0.65),  # False positive
    ((50, 50, 100, 100), 0.55),    # False positive
]

ground_truths = [(100, 100, 200, 200), (300, 300, 400, 400)]

precisions, recalls = precision_recall_curve(predictions_with_conf, ground_truths)

plt.figure(figsize=(8, 6))
plt.plot(recalls, precisions, 'b-o', linewidth=2, markersize=8)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.xlim([0, 1.05])
plt.ylim([0, 1.05])
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate AP (area under curve, simplified)
ap = np.trapz(precisions, recalls)
print(f"Average Precision (AP): {ap:.4f}")

## Question 3.1

Given these detection results, calculate:
1. Precision and Recall at confidence threshold 0.5
2. mAP (using simplified trapezoid integration)

In [ ]:
# YOUR CODE HERE

# Given data
detections = [
    ((50, 50, 150, 150), 0.9, 'dog'),
    ((200, 200, 300, 300), 0.8, 'cat'),
    ((55, 55, 155, 155), 0.7, 'dog'),  # Duplicate
    ((400, 400, 500, 500), 0.6, 'dog'),  # False positive
    ((210, 210, 310, 310), 0.5, 'cat'),  # Duplicate
    ((600, 600, 700, 700), 0.4, 'bird'),  # False positive
]

ground_truth = [
    ((50, 50, 150, 150), 'dog'),
    ((200, 200, 300, 300), 'cat'),
    ((350, 350, 450, 450), 'dog'),  # Missed
]

# Calculate metrics
# ...

---

# Challenge Problems

## Challenge 1: Soft-NMS

Instead of completely removing overlapping boxes, Soft-NMS reduces their confidence:

```
For each box with IoU > threshold with best box:
  score = score * (1 - IoU)  # Linear decay
  # or
  score = score * exp(-IoU^2 / sigma)  # Gaussian decay
```

Implement Soft-NMS and compare with standard NMS.

In [ ]:
# YOUR CODE HERE

def soft_nms(boxes, scores, sigma=0.5, threshold=0.001):
    """
    Soft Non-Maximum Suppression with Gaussian decay.
    """
    pass

## Challenge 2: Real-time Detection

Create a script that runs YOLO on webcam feed in real-time.
Display FPS (frames per second) on the video.

In [ ]:
# YOUR CODE HERE

# Note: This won't work in Colab, but will work locally

def webcam_detection():
    """
    Run YOLO detection on webcam feed.
    """
    pass

## Challenge 3: Multi-Object Tracking

Extend detection to tracking: assign consistent IDs to objects across frames.

Hint: Use IoU between frames to associate detections.

In [ ]:
# YOUR CODE HERE

def simple_tracker(prev_detections, curr_detections, iou_threshold=0.3):
    """
    Simple IoU-based tracker.
    
    Returns:
        List of (detection, track_id) pairs
    """
    pass

---

# Summary

## What We Learned

| Concept | Description |
|---------|-------------|
| **NMS** | Remove duplicate detections by IoU |
| **YOLO Format** | Normalized (cx, cy, w, h) labels |
| **Fine-tuning** | Train pre-trained model on custom data |
| **Precision** | TP / (TP + FP) - accuracy of detections |
| **Recall** | TP / (TP + FN) - completeness of detections |
| **mAP** | Average precision across classes and IoU thresholds |

## Key Code Patterns

```python
# Fine-tune YOLO
model = YOLO('yolov8n.pt')
model.train(
    data='data.yaml',
    epochs=50,
    imgsz=640,
    batch=16
)

# Evaluate
metrics = model.val()
print(f"mAP50: {metrics.box.map50}")

# NMS
keep_indices = nms(boxes, scores, iou_threshold=0.5)
```

## Congratulations!

You've completed the Object Detection labs! You now understand:
- How YOLO works and how to use it
- How to prepare custom datasets
- How to fine-tune models for your specific use case
- How to evaluate detection performance